# PGs generation, detection & recognition via learning delays

In [99]:
from brian2 import *
%matplotlib inline
from brian2 import SpikeGeneratorGroup
from brian2 import NeuronGroup
import numpy as np
import random
import pandas as pd

On s'intéresse ici à la génération, la détection et l'apprentissage de patterns temporels grâce à l'apprentissage des délais de manière à ce qu'un pattern temporel d'intéret s'articule en groupe polychrone (PG). Un groupe polychrone est définit par un groupe de neurone qui déchargent de manière asycnhrone, à différents moments, mais qui, grâce à leurs délais, transmettent l'information à un neurone post synaptique de façon sychrone.

L'idée ici est dans un premier temps de réaliser un modèle génératif de pattern temporel et un modèle de détection de groupes polychrones en Brian. Pour celà, on utilise un réseau de neurone à spike de trois couches : 
- la première couche est constituée de neurones du SpikeGeneratorGroup A et sert à la génération.
- la deuxième couche est constituée de neurones appartenant au NeuronGroup B et sert à la génération. 
- la troisième couche est constituée de neurones appartenant au NueronGroup C et sert à la détection.

Les neurones de la couche A vont émettrent un spike à un moment donné de la simulation. Chaque neurone de cette couche projette sur au moins trois neurones de la couche B, selon un certain poid et un certain delais. Si un neurone de la couche A spike, alors les neurones de la couche B sur lesquels il projette vont emettrent un spike (en fonction de leur poid et de leur délai). Tous les neurones de la couche B qui déchargent en réponse au spike du neurone de la couche A consitituent un groupe polychrone. 
Avec cette organisation, on génère un rasterplot artificiel (actvité des neurones de la couche B) dans lequel on voudrait détecter des groupes polychrones. Les spikes appartenant à un même groupe sont déterminés en fonction du neurone de la couche A qui a engendré leur décharge. L'activité de la couche B correspond donc à notre entrée et l'activité des neurones de la couche A correspond à notre ground truth, ce que l'on voudrait détecter. Un spike d'un neurone dans la couche A correspond à l'occurence d'un groupe polychrone. 
Puisque l'on connait les connections a->b, les poids et les délais, il est facile d'organiser un groupe de neurone en groupe polychrones en réalisant une troisièmpe couche, équivalente à la couche a. On construit des connections b->c de la même manière que a->b, avec les mêmes poids. On ajuste les délais de sorte à ce que les spikes d'un groupe de neurones polychrone arrivent de façon synchrone sur un neurone de la couche c et induisent leur décharge. Lorsqu'un neurone c spike ça veut dire que les neurones projettant sur lui spike avec une certaine séquence temporelle. On detecte donc une séquence temporelle d'intéret à un moment dans le temps. En récupérant les délais on peut connaitre cette séquence temporelle. 

### variable definition

In [100]:
Ni = 5 #nb de PGs différents
Nj = 10 #nb de N
n_pattern = 100 # nb d'occurrence des PGs 
duration = 10000*msecond

v_r = 1e-322*mV 
tau = 0.0001*second

PGs_pattern = {}
PGs_id_tps = {}
detection = {}
state = {}

a = np.arange(Ni)
cmap = plt.cm.get_cmap("plasma")
color_dict = pd.Series({i:cmap(i/len(a)) for i,k in enumerate(a)})

In [103]:
# --- def du moment d'occurence des PGs -------------------------------------------------------------------------------------------------

i_indices = np.random.randint(0, Ni, size = n_pattern) # nombre de PG observé (n_pattern), de Ni sortes différentes
i_temps = np.random.uniform(0, duration, size = n_pattern)*second # temps d'occurence des n_pattern PG 


# --- def des projections des neurones pré-syn (i.e. des des PGs) -----------------------------------------------------------------------

i_syn=[]
n_syn = []
W = []

for k in range(Ni) : 
    n_j = np.random.randint(3, Nj, size = 1) # nombre de neurone qu'un Ni va connecter : au moins 3 neurones impliqués dans un PG
    i_syn.append(random.sample(range(Nj), int(n_j))) # def des j connectés aux i, pas de repetition (pas de delais heterosynaptique)
    n_syn.append(len(i_syn[k])) # def du nb de synapses pour set des poids et délais aléatoires, voir ci-après 
    W.append(np.random.rand(int(n_j)))
    W[k] /= sum(W[k])
    
    
n_syn = sum(n_syn) 


# --- def des poids et delais synaptiques -----------------------------------------------------------------------------------------------

weight = np.random.rand(n_syn) # des fois les poids générés pour 1 gp sont trop faibles pour que la detection marche, faudrait il faire en sorte que la somme des poids générés pour 1 gp soit = 1 ? 
delay = np.random.rand(n_syn)*0.1*second # là entre 0 et 100 -> 144 

In [104]:
sum(W[0])

0.9999999999999998

### NN simulation for PGs generation and detection

In [ ]:
# program to compute the time
# of execution of any python code
import time
 
# we initialize the variable start
# to store the starting time of
# execution of program
start = time.time()

start_scope()
for i in range(n_pattern) :
    
    a = SpikeGeneratorGroup(Ni, [i_indices[i]], [i_temps[i]/ms*msecond])
    a_spike= SpikeMonitor(a)
    
    
    b = NeuronGroup(Nj, 
                    ''' dv/dt = -v/tau : volt
                        tau : second''',
                    threshold= 'v > 0.02*volt',
                    reset= 'v = v_r')
    b.v = v_r
    b.tau = tau
    b_spike = SpikeMonitor(b)
    

    s = Synapses(a,b, on_pre='v+=(0.01*volt*w)', model = 'w:1')
    
    for k in range(Ni):
        s.connect(i = k , j = i_syn[k])
        s.w[k,:] = W[k]
    s.delay[:,:] = delay
    
    
    c =  NeuronGroup(Ni, 
                    ''' dv/dt = -v/tau : volt
                        tau : second''',
                    threshold= 'v > 0.01*volt',
                    reset= 'v = v_r',
                    method = 'exact')
    c.v = v_r
    c.tau = tau
    c_spike = SpikeMonitor(c)
    
    syn = Synapses(b,c, on_pre='v+=(0.01*volt*w)', model = 'w:1')
    
    for k in range(Ni):
        syn.connect(i = i_syn[k], j = k)         
        syn.w[:,k] = W[k]
        
    for toto in range(Ni) :
        syn.delay[:,[toto]] = max(s.delay[toto,:])-s.delay[toto,:]
    
    net_g = Network(collect())
    net_g.add(a, a_spike, b, b_spike, c, c_spike, s, syn)
    net_g.run(duration) 

    PGs_id_tps[i] = (a_spike.t, a_spike.i) # je vourdrais avoir l'echelle de temps en ms ezt non en s 
    PGs_pattern[i] = (b_spike.t, b_spike.i)
    detection[i] = (c_spike.t, c_spike.i) # -max(syn.delay[:,[c_spike.i]]) pour que ce soit le premier spike que l'on detecte, peut etre pas essentiel
    
# now we have initialized the variable
# end to store the ending time after
# execution of program
end = time.time()
 
# difference of start and end variables
# gives the time of execution of the
# program in between
print("The time of execution of above program is :", end-start)

In [ ]:
def visualise_connectivity(s): # ajouter les delays
    Ns = len(s.source)
    Nt = len(s.target)
    figure(figsize=(15,8))
    
    subplot(141)
    plot(zeros(Ns), arange(Ns), 'ok', ms=7)
    plot(ones(Nt), arange(Nt), 'ok', ms=7)
    for i, j in zip(s.i, s.j):
        plot([0, 1], [i, j], '-k')
    xticks([0, 1], ['Source', 'Target'])
    ylabel('Neuron index')
    xlim(-0.1, 1.1)
    ylim(-1, max(Ns, Nt))
    
    subplot(142)
    plot(s.i, s.j, 'ok')
    xlim(-1, Ns)
    ylim(-1, Nt)
    xlabel('Source neuron index')
    ylabel('Target neuron index')
    
    subplot(143) 
    scatter(s.i, s.j, s.w*30 )
    xlabel('Source neuron index')
    ylabel('Target neuron index')
    
    subplot(144) 
    scatter(s.i, s.j, s.delay*300)
    xlabel('Source neuron index')
    ylabel('Target neuron index')

In [ ]:
visualise_connectivity(s)

In [ ]:
visualise_connectivity(syn)

In [ ]:
plt.figure(figsize=(10,4))
for i in range(n_pattern) :
    plt.scatter(PGs_id_tps[i][0], PGs_id_tps[i][1], color = color_dict[i_indices[i]], marker = "|")
    xlabel('Time (s)')
    ylabel('PGs')
    title('occurence of PGs')

In [ ]:
id = []
plt.figure(figsize=(10,4))
for i in range(n_pattern) :
    plt.scatter(PGs_pattern[i][0], PGs_pattern[i][1], color = color_dict[i_indices[i]], marker = "|")
    id.append(PGs_pattern[i][1])
    xlabel('Time (s)')
    ylabel('Neuron index');

In [ ]:
plt.figure(figsize=(10,4))
for i in range(n_pattern) :
    plt.scatter(detection[i][0], detection[i][1], color = color_dict[i_indices[i]], marker = "|")
    xlabel('Time (s)')
    ylabel('PGs')
    title('occurence of PGs')

ici, ce sont les connections b->c qui déterminent les neurones impliqués dans un PG, les poids sont donc obsolètes pour la détection des PGs. Ils décervent même un peu, par exemple pour la détection du PG 2, la somme des poids est faible et donc la synchronisation des décharges des neurones le composant ne permet pas de dépasser le seuil de 0.02, j'ai du l'abaisser à 0.01. 
Il faudrait plutot faire une couche de détections où tous les b connectent tous les c et où les poids sont importants pour les neurones où a->c existe et faibles pour les connections où a->c n'existe pas. (ici b->c n'existe que si a->c existe, les poids ne servent donc à rien)

# supervised learning of weight and delay for recognition of PGs

l'idée serait d'apprendre dans un premier temps les poids, pour selectionner les neurones impliqués dans une séquence temporelle. Ensuite, on apprendrait les délais necessaires pour synchroniser les neurones de ce groupe. En récupérant les poids on pourrait connaitre les neurones impliqués dans un groupe et en récupérant les délais necessaire à la synchronisation, on pourrait connaitre la séquence temporelle qu'ils constituent.

On pourrait aussi envisager de ne faire qu'un apprentissage des delais ?

L'idée que j'ai là, serait de faire une couche y qui a la même activité que c. Elle connecte tous les neurones de la couche x, qui elle, à l'activité de B. On met une règle de STDP des délais entre les deux. Si c spike, alors il faut synchroniser les spikes venant dans les 100ms avant. 
Ensuite, renforcer le poid des neurones synchronisés. 


In [ ]:
# obtenir le temps et l'indice de chaque spike, dans l'ordre d'apparition, indépendemment du PGs auquel il appartient, en grandeur ms (mais sans unité de temps) et int
all_spike_time_x = []

for k in range(n_pattern) :
    for i in range(len(PGs_pattern[k][1])):
        all_spike_time_x.append(tuple((round(PGs_pattern[k][0][i]*1000/second), PGs_pattern[k][1][i])))
        
all_spike_time_x.sort(key=lambda y: y[0]) #pour trier de tmin à tmax

In [ ]:
all_spike_time_y = []

for k in range(n_pattern) :
        all_spike_time_y.append(tuple((np.round(detection[k][0][0]*1000/second), detection[k][1][0])))
        
all_spike_time_y.sort(key=lambda y: y[0]) #pour trier de tmin à tmax

In [ ]:
detection

In [ ]:
temps_x = []
ind_x = []
temps_y = []
ind_y = []

for i in range(len(all_spike_time_x)): 
    temps_x.append(all_spike_time_x[i][0]*ms)
    ind_x.append(all_spike_time_x[i][1])
    
for i in range(len(all_spike_time_y)): 
    temps_y.append(all_spike_time_y[i][0]*ms)
    ind_y.append(all_spike_time_y[i][1])

In [ ]:
taupre = taupost = 20*ms
dmax = 100*ms
Apre = dmax
Apost = -Apre*0 # là je met = 0 mais ce serait peut etre mieux de carrément supprimer la partie où delta t < 0
delta_t = linspace(-100, 100, 200)*ms

W = where(delta_t>0, Apre*exp(delta_t/taupre), Apost*exp(-delta_t/taupost))
plot(delta_t/ms, W)
xlabel(r'$\Delta t$ (ms)')

ylabel('d')
axhline(0, ls='-', c='k');

In [ ]:
start_scope()

taupre = taupost = 20*ms
dmax = 0.1*second
Apre = 0.001*second
Apost = -Apre*taupre/taupost*1.05

G = NeuronGroup(2, 'v:1', threshold='t>(1+i)*5*ms', refractory=100*ms)

S = Synapses(G, G,
             '''
             d : second
             dapre/dt = -apre/taupre : second (clock-driven)
             dapost/dt = -apost/taupost : second (clock-driven)
             ''',
             on_pre='''
             delay += d
             apre += Apre
             d = clip(d+apost, 0, dmax)
             ''',
             on_post='''
             delay += d
             apost += Apost
             d = clip(d+apre, 0, dmax)
             ''', method='linear')
S.connect(i=0, j=1)
S.delay[:,:]= 1*ms
M = StateMonitor(S, ['d', 'apre', 'apost'], record=True)
V = StateMonitor(G, ['v'], record=True)
run(30*ms)

figure(figsize=(4, 8))
subplot(311)
plot(M.t/ms, M.apre[0], label='apre')
plot(M.t/ms, M.apost[0], label='apost')
legend()
subplot(312)
plot(M.t/ms, M.d[0], label='w')
legend(loc='best')
xlabel('Time (ms)');
subplot(313)
plot(V.t/ms, V.v[1], label='vpost')

In [30]:
start_scope()

taupre = taupost = -20*ms
dmax = 100
apre_max = 0.0001
Apre = 0.01
Apost = 0
defaultclock.dt = 0.0001*ms

x = SpikeGeneratorGroup(Nj, ind_x, temps_x)
y = SpikeGeneratorGroup(Ni, ind_y, temps_y)

naps = Synapses(x,y, 
             '''
             d : 1
             dapre/dt = -apre/taupre : 1 (event-driven)
             dapost/dt = -apost/taupost : 1 (event-driven)
             ''',
             on_pre=''' 
             apre += Apre
             d = clip(d+apost, 0, dmax)
             delay += d
             ''',
             on_post='''
             apost += Apost
             d = clip(d+apre, 0, dmax)
             delay += d
             ''')

naps.connect(p=1)
naps.delay[:,:] =  np.ones((Ni*Nj))*ms
M = StateMonitor(naps, ['d', 'apre', 'apost',], record=True)

run(duration)

figure(figsize=(4, 8))
subplot(211)
plot(M.t/ms, M.apre[0], label='apre')
plot(M.t/ms, M.apost[0], label='apost')
legend()
subplot(212)
plot(M.t/ms, M.d[0], label='d')
legend(loc='best')
xlabel('Time (ms)');

NameError: name 'ind_x' is not defined

In [ ]:
figure(figsize=(4, 8))
subplot(211)
plot(M.t/ms, M.apre[0], label='apre')
plot(M.t/ms, M.apost[0], label='apost')
legend()
subplot(212)
plot(M.t/ms, M.d[0], label='w')
legend(loc='best')
xlabel('Time (ms)');

In [ ]:
def visu_connectivity(S):
    Ns = len(S.source)
    Nt = len(S.target)
    figure(figsize=(10, 4))
    subplot(131)
    plot(zeros(Ns), arange(Ns), 'ok', ms=10)
    plot(ones(Nt), arange(Nt), 'ok', ms=10)
    for i, j in zip(S.i, S.j):
        plot([0, 1], [i, j], '-k')
    xticks([0, 1], ['Source', 'Target'])
    ylabel('Neuron index')
    xlim(-0.1, 1.1)
    ylim(-1, max(Ns, Nt))
    subplot(132)
    plot(S.i, S.j, 'ok')
    xlim(-1, Ns)
    ylim(-1, Nt)
    xlabel('Source neuron index')
    ylabel('Target neuron index')
    subplot(133) 
    scatter(S.i, S.j, S.delay*300)
    xlabel('Source neuron index')
    ylabel('Target neuron index')

In [ ]:
visu_connectivity(naps)

In [ ]:
visu_connectivity(syn)

In [ ]:
visu_connectivity(s)

# unsupervised recognition of PGs

## detection of temporal patterns 

In [ ]:
all_spike_time

In [ ]:
temps_tot

In [ ]:
# def de ma fenetre temporelle pour reconnaître les PGs
temps_tot = int(duration/msecond)
t_window = 200 #ms
nb_wind = int(temps_tot/t_window)
X = np.zeros((nb_wind, Nj, t_window))

In [ ]:
for k in range(nb_wind) :
    for t,i in (all_spike_time) : 
        if t<t_window : 
            X[1][i][t] = 1 #on peut faire [1,i,t]
            print('ok')
        if t_window*(k-1)<t<t_window*k : 
            X[k][i][t-t_window*(k-1)] = 1 
            print('okk')

In [ ]:
X[

In [ ]:
plot(X[58].T)

In [ ]:
all_spike_time

## cam's k-means